In [1]:
import numpy as np
import pandas as pd
import re
import os

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

from typing import List

#### Merge all the data

In [2]:
data0 = pd.read_csv('data/house_data0.csv')
data0.head()

,title,address,bed,bath,toilet,pkn_space,price
0,4 bedroom detached duplex for sale,"Megamound Estate, Lekki County Homes., Ikota, ...",4Bedrooms,5Bathrooms,5Toilets,Save,"75,000,000"
1,4 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",4Bedrooms,4Bathrooms,5Toilets,2Parking Spaces,"47,000,000"
2,5 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",5Bedrooms,4Bathrooms,6Toilets,4Parking Spaces,"80,000,000"
3,3 bedroom detached duplex for sale,"Ikola Road, By Command., Alagbado, Ifako-Ijaiy...",3Bedrooms,Save,NaN,NaN,"27,500,000"
4,3 bedroom terraced duplex for sale,"2nd Tollgate, Lekki, Lagos",3Bedrooms,3Bathrooms,4Toilets,Save,"45,000,000"


In [3]:
data0.shape

(1050, 7)

In [4]:
files = [file for file in os.listdir('data/')]

files

['house_data0.csv', 'house_data1.csv', 'house_data2.csv', 'house_data3.csv']

In [5]:
all_df = pd.DataFrame()  # empty dataframe

print(f"Before concatenatiion: {all_df.shape} \n")

for file in files:
    df = pd.read_csv(f'data/{file}')
    all_df = pd.concat([all_df, df], axis='index')
    
print(f"After concatenatiion: {all_df.shape} \n")

Before concatenatiion: (0, 0) 

After concatenatiion: (7100, 7) 



In [6]:
all_df.head()

,title,address,bed,bath,toilet,pkn_space,price
0,4 bedroom detached duplex for sale,"Megamound Estate, Lekki County Homes., Ikota, ...",4Bedrooms,5Bathrooms,5Toilets,Save,"75,000,000"
1,4 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",4Bedrooms,4Bathrooms,5Toilets,2Parking Spaces,"47,000,000"
2,5 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",5Bedrooms,4Bathrooms,6Toilets,4Parking Spaces,"80,000,000"
3,3 bedroom detached duplex for sale,"Ikola Road, By Command., Alagbado, Ifako-Ijaiy...",3Bedrooms,Save,NaN,NaN,"27,500,000"
4,3 bedroom terraced duplex for sale,"2nd Tollgate, Lekki, Lagos",3Bedrooms,3Bathrooms,4Toilets,Save,"45,000,000"


### Clean the data.

In [7]:
# adjust pandas display settings
pd.set_option('max_colwidth', 100)

all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7100 entries, 0 to 3599
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      7100 non-null   object
 1   address    7100 non-null   object
 2   bed        7055 non-null   object
 3   bath       7100 non-null   object
 4   toilet     6590 non-null   object
 5   pkn_space  6450 non-null   object
 6   price      7100 non-null   object
dtypes: object(7)
memory usage: 443.8+ KB


In [28]:
def get_address(addr: str) -> str:
    """ 
    ============================================================
        Extract the address.
    """
    result = addr.split(',')[-2:]  # select the last two elements
    result = [x.strip() for x in result]  # remove the white spaces
    result = ', '.join(result)  # join on spaces (no longer a list)
    return result


def clean_text(text: str) -> str:
    """
    ============================================================
        Clean the text.
    """
    pattern = r"\D+"  # non-digits
    result = re.sub(pattern, '', text, flags=re.I)
    return result


def clean_price(text: str) -> int:
    """
    ============================================================
        Clean the price and return an integer.
    """
    pattern = r","  # non-digits
    result = re.sub(pattern, '', text, flags=re.I)
    return int(result)


In [54]:
### drop the missing values
all_df1 = all_df.copy()
all_df1 = all_df1.dropna()

In [55]:
# clean the data
all_df1['address'] = all_df1['address'].apply(get_address)
all_df1['bed'] = all_df1['bed'].apply(clean_text)
all_df1['bath'] = all_df1['bath'].apply(clean_text)
all_df1['toilet'] = all_df1['toilet'].apply(clean_text)
all_df1['pkn_space'] = all_df1['pkn_space'].apply(clean_text)
all_df1['price'] = all_df1['price'].apply(clean_price)

# conert to numeric data type
for col in ['bed', 'bath', 'toilet', 'pkn_space']:
    all_df1[col] = pd.to_numeric(all_df1[col])


all_df1.head()

,title,address,bed,bath,toilet,pkn_space,price
0,4 bedroom detached duplex for sale,"Lekki, Lagos",4,5,5,NaN,75000000
1,4 bedroom semi-detached duplex for sale,"Ajah, Lagos",4,4,5,2.0,47000000
2,5 bedroom semi-detached duplex for sale,"Ajah, Lagos",5,4,6,4.0,80000000
4,3 bedroom terraced duplex for sale,"Lekki, Lagos",3,3,4,NaN,45000000
5,5 bedroom detached duplex for sale,"Lekki, Lagos",5,5,6,NaN,165000000


In [57]:
all_df1.describe()

,bed,bath,toilet,pkn_space,price
count,6450.000000,6450.000000,6450.000000,4520.000000,6.450000e+03
mean,4.365891,4.586822,6.494574,44.486726,1.882789e+08
std,1.064409,1.734595,30.371689,215.509026,5.205046e+08
min,1.000000,1.000000,1.000000,1.000000,1.200000e+06
25%,4.000000,4.000000,5.000000,3.000000,6.000000e+07
50%,4.000000,4.500000,5.000000,4.000000,9.000000e+07
75%,5.000000,5.000000,6.000000,6.000000,1.800000e+08
max,13.000000,47.000000,1000.000000,3600.000000,1.650000e+10


In [ ]:
clean_price()